<a href="https://colab.research.google.com/github/Wakana-the-cafehopper/Research/blob/main/Can_digital_trade_power_energy_inclusion_202507.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

# load the excel file
xls = pd.ExcelFile("/content/Master dataset_Can Digital Trade Power Energy Inclusion_202507.xlsx")
print(xls.sheet_names) # check available sheets

['Data', 'solar_imports', 'ecomm_readiness', 'mobile_money, digital_platform_']


In [6]:
solar_df = xls.parse('solar_imports')
solar_clean = solar_df[
    (solar_df['reporterDesc'].isin(['Indonesia', 'Philippines', 'Viet Nam'])) &
    (solar_df['refYear'].between(2015, 2022)) &
    (solar_df['cmdCode'] == 854140)
][['reporterDesc', 'refYear', 'primaryValue', 'Source']]

solar_clean = solar_clean.rename(columns={
    'reporterDesc': 'Country',
    'refYear': 'Year',
    'primaryValue': 'SolarImports',
    'Source': 'Source'
})

In [7]:
findex_df = xls.parse('mobile_money, digital_platform_')

findex_clean = findex_df[[
    'Country name', 'Year',
    'Mobile money account (% age 15+)',
    'Used a mobile phone or the internet to buy something online (% age 15+)',
    'Source',
]].rename(columns={
    'Country name': 'Country',
    'Mobile money account (% age 15+)': 'MobileMoney',
    'Used a mobile phone or the internet to buy something online (% age 15+)': 'DigitalPlatformUsage',
    'Source': 'Source'
})

# normalize country name
findex_clean['Country'] = findex_clean['Country'].replace({'Vietnam': 'Viet Nam'})

In [8]:
ecomm_df = xls.parse('ecomm_readiness')

ecomm_clean = ecomm_df[['CountryName', 'IndicatorValue', 'Source']]

ecomm_clean = ecomm_clean.rename(columns={
    'CountryName': 'Country',
    'IndicatorValue': 'EcommReadiness',
})

ecomm_clean['Year'] = 2015

In [9]:
core_df = xls.parse('Data')

# reshape to panel format
data_trimmed = core_df.iloc[:, :23]

data_long = pd.melt(
    data_trimmed,
    id_vars=['Variable Name', 'Country Name', 'Country Code'],
    var_name='Year',
    value_name='Value'
)

# Replace '..' with NaN in the 'Value' column
data_long['Value'] = data_long['Value'].replace('..', pd.NA)

# Convert 'Value' column to numeric, coercing errors to NaN
data_long['Value'] = pd.to_numeric(data_long['Value'], errors='coerce')


# Extract year and filter out rows where year extraction failed
data_long['Year'] = data_long['Year'].str.extract(r'(\d{4})')
data_long = data_long.dropna(subset=['Year'])
data_long['Year'] = data_long['Year'].astype(int)


data_pivot = data_long.pivot_table(
    index=['Country Name', 'Year'],
    columns='Variable Name',
    values='Value'
).reset_index()

data_pivot['Country Name'] = data_pivot['Country Name'].replace({'Vietnam': 'Viet Nam'})

data_pivot = data_pivot.rename(columns={
    'Country Name': 'Country',
    'rural_elec_access': 'RuralElecAccess',
    'broadband_penetration': 'BroadbandPenetration',
    'customs_performance': 'CustomsPerformance',
    'gdp_per_capita': 'GDPperCapita'
})

# normalize country name
data_pivot['Country'] = data_pivot['Country'].replace({'Vietnam': 'Viet Nam'})

# previwe results
data_pivot.head()

Variable Name,Country,Year,BroadbandPenetration,CustomsPerformance,GDPperCapita,RuralElecAccess
0,Indonesia,2010,10.9,2.430000,2670.812910,89.4
1,Indonesia,2011,12.3,NaN,2799.624688,90.2
2,Indonesia,2012,14.5,2.530000,2930.518445,92.6
3,Indonesia,2013,14.9,NaN,3055.242460,93.0
4,Indonesia,2014,17.1,2.869565,3170.721030,94.0


In [10]:
# strip whitespace and unify country names
for df in [solar_clean, findex_clean, ecomm_clean, data_pivot]:
  df['Country'] = df['Country'].str.strip()
  df['Country'] = df['Country'].replace({'Vietnam': 'Viet Nam'})

# ensure Year is integer in all dataframes
for df in [solar_clean, findex_clean, ecomm_clean, data_pivot]:
  df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# drop rows where Country or Year is missing
for df in [solar_clean, findex_clean, ecomm_clean, data_pivot]:
  df.dropna(subset=['Country', 'Year'], inplace=True)

# try merging again
from functools import reduce

dfs = [solar_clean, findex_clean, ecomm_clean, data_pivot]
master_df = reduce(lambda left, right: pd.merge(left, right, on=['Country', 'Year'], how='outer'), dfs)
master_df = master_df.sort_values(by=['Country', 'Year']).reset_index(drop=True)

# display to verify
display(master_df.head())

,Country,Year,SolarImports,Source_x,MobileMoney,DigitalPlatformUsage,Source_y,EcommReadiness,Source,BroadbandPenetration,CustomsPerformance,GDPperCapita,RuralElecAccess
0,Indonesia,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.9,2.430000,2670.812910,89.4
1,Indonesia,2011,NaN,NaN,NaN,NaN,https://www.worldbank.org/en/publication/globa...,NaN,NaN,12.3,NaN,2799.624688,90.2
2,Indonesia,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,2.530000,2930.518445,92.6
3,Indonesia,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.9,NaN,3055.242460,93.0
4,Indonesia,2014,NaN,NaN,0.004496,NaN,https://www.worldbank.org/en/publication/globa...,NaN,NaN,17.1,2.869565,3170.721030,94.0


In [11]:
master_df.to_excel("Cleaned_Merge_Master.xlsx", index=False)